In [1]:
# import pandas as pd
import MagmaPandas as mp
import MagmaPandas.geochemistry.PEC as pec
import pandas as pd
import elements as e
import matplotlib.pyplot as plt
import meltInc.plotting as p

p.layout()
config = pec.PEC_configuration()
config.print()

############## Post-entrapment modification #############
#################### correction model ###################

Settings________________________________________________
Stepsize equilibration (moles).....................0.002 
Stepsize crystallisation (moles)...................0.05  
Decrease factor....................................5     
FeO convergence (wt. %)............................0.05  
Kd convergence.....................................0.0001
Temperature convergence (°)........................0.1   


In [2]:
melt = mp.read_melt_inclusion('./melt.csv', index_col=['name'], total_col='total')
olivine = mp.read_olivine('./olivine.csv', index_col=['name'], total_col="total")

In [3]:
forsterites = olivine.forsterite
inclusions = melt.copy()
ol_host = olivine.copy()
Fo = 0.9
# FeO_initial = 11.2
# FeO_initial = lambda x: 11.3981 - 0.07896 * x["MgO"]
FeO_initial = 11.2
P_bar = 1

In [ ]:
equilibrated_inclusions = mp.MagmaFrame(index=inclusions.index, columns=inclusions.columns, dtype=float, units="wt. %", datatype="oxide")
ol_equilibrated = pd.Series(index=inclusions.index, dtype=float)

totalItems= equilibrated_inclusions.shape[0]
for i, ((idx, mi), (_, ol)) in enumerate(zip(inclusions.iterrows(), ol_host.iterrows())):

    print(f"{i + 1:0>3}/{totalItems:0>3} inclusions equilibrated", end= "\r")
    x, oc, kd = pec.Fe_equilibrate(mi, ol, P_bar)
    equilibrated_inclusions.loc[idx] = x.iloc[-1]
    ol_equilibrated.loc[idx] = oc[-1]

In [ ]:
corrected_inclusions = mp.Melt(index=inclusions.index, columns=inclusions.columns, dtype=float, units="wt. %", datatype="oxide")
ol_crystallised = pd.Series(index=inclusions.index, dtype=float)

totalItems = equilibrated_inclusions.shape[0]

for i, ((idx, mi), (_, ol)) in enumerate(zip(equilibrated_inclusions.iterrows(), ol_host.iterrows())):

    print(f"{i + 1:0>3}/{totalItems:0>3} inclusions corrected", end= "\r")

    x2, oc2, kd2, t = pec.crystallisation_correction(mi, ol, FeO_initial, P_bar)
    corrected_inclusions.loc[idx] = x2.iloc[-1]
    ol_crystallised.loc[idx] = oc2[-1]

In [4]:
mi_pec = pec.PEC_olivine(inclusions, ol_host, P_bar, FeO_initial)

In [5]:
mi_pec.reset()
x, oc, t = mi_pec.correct()

029/029 inclusions equilibrated
029/029 inclusions corrected


In [6]:
oc

name
PI032-01-01    0.122860
PI032-01-04    0.061500
PI032-02-01    0.119120
PI032-02-02    0.110184
PI032-02-03    0.125900
PI032-03-01    0.119916
PI032-04-01    0.111156
PI032-04-02    0.119364
PI032-04-03    0.104304
PI032-05-01    0.032192
PI041-02-01    0.011860
PI041-02-02    0.040268
PI041-03-01    0.138420
PI041-03-03    0.129248
PI041-04-02    0.040380
PI041-04-03    0.045240
PI041-04-04    0.027000
PI041-04-05    0.042148
PI041-04-06    0.044780
PI041-04-07    0.057896
PI041-05-02    0.009200
PI041-05-03    0.005560
PI041-05-04    0.014220
PI041-05-06    0.010540
PI041-07-01    0.093976
PI041-07-02    0.096228
PI041-07-06    0.096892
PI052-01-02   -0.015808
PI052-02-01    0.104500
dtype: float64

In [ ]:
t = mi_pec.correct_inclusion(5)

In [ ]:
markersize = 12

fig, ax = plt.subplots(figsize=(8,7))

for (idx, i), (_, j) in zip(mi_pec.inclusions.iterrows(), mi_pec.inclusions_uncorrected.iterrows()):
    data = pd.concat([pd.Series(i), pd.Series(j)], axis=1).T
    data.index = ["corrected", "original"]
    plt.plot(data["MgO"], data["FeO"], "--", color="k")

plt.plot(mi_pec.inclusions["MgO"], mi_pec.inclusions["FeO"], "o", markersize=markersize)
plt.plot(mi_pec.inclusions_uncorrected["MgO"], mi_pec.inclusions_uncorrected["FeO"], "D", markersize=markersize)

ax.set_xlabel("wt. % MgO")
ax.set_ylabel("wt. % FeO")

plt.show()